In [1]:
import itertools

import numpy as np
import IPython.display as disp
from sympy import simplify, collect, log, I, diff, eye, solve
from symengine import expand, Symbol, conjugate, symbols, Matrix, linsolve, Add

# Normal Form of the Generalised Swift-Hohenberg Equation
In this notebook we calculate the coefficients of the normal form of the generalised Swift-Hohenberg equation

In [2]:
# linear matrix
L = Matrix([[0, 1, 0, 0, 0, 0, 0, 0],
            [0, 0, 1, 0, 0, 0, 0, 0],
            [0, 0, 0, 1, 0, 0, 0, 0],
            [0, 0, 0, 0, 1, 0, 0, 0],
            [0, 0, 0, 0, 0, 1, 0, 0],
            [0, 0, 0, 0, 0, 0, 1, 0],
            [0, 0, 0, 0, 0, 0, 0, 1],
            [-1, 0, -4, 0, -6, 0, -4, 0]])

In [3]:
L

[0, 1, 0, 0, 0, 0, 0, 0]
[0, 0, 1, 0, 0, 0, 0, 0]
[0, 0, 0, 1, 0, 0, 0, 0]
[0, 0, 0, 0, 1, 0, 0, 0]
[0, 0, 0, 0, 0, 1, 0, 0]
[0, 0, 0, 0, 0, 0, 1, 0]
[0, 0, 0, 0, 0, 0, 0, 1]
[-1, 0, -4, 0, -6, 0, -4, 0]

In [4]:
# reversibility
R = eye(8)
for i in range(1, 5):
    R[2 * i - 1, 2 * i - 1] = -1

In [5]:
R

Matrix([
[1,  0, 0,  0, 0,  0, 0,  0],
[0, -1, 0,  0, 0,  0, 0,  0],
[0,  0, 1,  0, 0,  0, 0,  0],
[0,  0, 0, -1, 0,  0, 0,  0],
[0,  0, 0,  0, 1,  0, 0,  0],
[0,  0, 0,  0, 0, -1, 0,  0],
[0,  0, 0,  0, 0,  0, 1,  0],
[0,  0, 0,  0, 0,  0, 0, -1]])

In [6]:
# main variables
A = Symbol('A')
B = Symbol('B')
C = Symbol('C')
D = Symbol('D')

A_bar = Symbol('A_bar')  # conjugate(A)
B_bar = Symbol('B_bar')  # conjugate(B)
C_bar = Symbol('C_bar')  # conjugate(C)
D_bar = Symbol('D_bar')  # conjugate(D)

z_tilde = Matrix([A, B, C, D, A_bar, B_bar, C_bar, D_bar])

relations = {conjugate(A): A_bar, conjugate(B): B_bar,
             conjugate(C): C_bar, conjugate(D): D_bar,
             conjugate(A_bar): A, conjugate(B_bar): B,
             conjugate(C_bar): C, conjugate(D_bar): D}

In [7]:
quadratic_terms = set(np.product(el) for el in itertools.product(z_tilde, z_tilde))
cubic_terms = set(np.product(el) for el in itertools.product(z_tilde, z_tilde, z_tilde))

In [8]:
# integrals
c1 = A * A_bar
c2 = I * B / A + log(A)
c3 = I * (A * B_bar - A_bar * B)
c4 = C / A - I * B * log(A) / A - (log(A))**2 / 2
c5 = A * C_bar - B * B_bar + A_bar * C
c6 = -I * D / A - C * log(A) / A + I * B * (log(A))**2 / (2 * A) + (log(A))**3 / 6
c7 = I * (A * D_bar - B * C_bar + B_bar * C - A_bar * D)
w1 = c2**2 + 2 * c4
w2 = c2**3 + 3 * c2 * c4 + 3 * c6

In [9]:
# vectors and matrices
zeta_0 = Matrix([1, I, -1, -I, 1, I, -1, -I])
zeta_1 = Matrix([0, 1, 2 * I, -3, -4 * I, 5, 6 * I, -7])
zeta_2 = Matrix([0, 0, 1, 3 * I, -6, -10 * I, 15, 21 * I])
zeta_3 = Matrix([0, 0, 0, 1, 4 * I, -10, -20 * I, 35])

# conjugate doesn't work on symengine Matrix
zeta_0_bar = Matrix([conjugate(el) for el in zeta_0])
zeta_1_bar = Matrix([conjugate(el) for el in zeta_1])
zeta_2_bar = Matrix([conjugate(el) for el in zeta_2])
zeta_3_bar = Matrix([conjugate(el) for el in zeta_3])

M = Matrix([[1, 0, 0, 0, 1, 0, 0, 0],
            [I, 1, 0, 0, -I, 1, 0, 0],
            [-1, 2 * I, 1, 0, -1,-2 * I, 1, 0],
            [-I, -3, 3 * I, 1, I, -3, -3 * I, 1],
            [1, -4 * I, -6, 4 * I, 1, 4 * I, -6, -4 * I],
            [I, 5, -10 * I, -10, -I, 5, 10 * I, -10],
            [-1, 6 * I, 15, -20 * I, -1, -6 * I, 15, 20 * I],
            [-I, -7, 21 * I, 35, I, -7, -21 * I, 35]])

M_inv = M.inv()

L0 = M_inv * L * M

In [10]:
# polynomial vector
powers = [(0, 1, 2, 3) for _ in range(8)]
poly_powers = [el for el in itertools.product(*powers) if sum(el) in (2, 3)]
poly = 8 * [0]
quadratic_coefficients = []
cubic_coefficients = []
for idx in poly_powers:
    for j in range(8):
        coeff = Symbol('psi_{}'.format(''.join([str(el) for el in (j,) + idx])))
        poly[j] += coeff * np.product([x**i for x, i in zip(z_tilde, idx)])
        if sum(idx) == 2:
            quadratic_coefficients.append(coeff)
        elif sum(idx) == 3:
            cubic_coefficients.append(coeff)
psi = Matrix(poly)

## Reversibility
Now we make use of the reversibility of the system to determine relations between some of the unknown coefficients

In [11]:
# define a scaling to help collect together terms
epsilon = Symbol('epsilon')
scalings = {A: epsilon * A, B: epsilon * B, C: epsilon * C, D: epsilon * D,
            A_bar: epsilon * A_bar, B_bar: epsilon * B_bar, C_bar: epsilon * C_bar, D_bar: epsilon * D_bar}

In [12]:
psi_scaled = expand(psi.subs(scalings))

In [13]:
reversibility = {A: A_bar, B: -B_bar, C: C_bar, D: -D_bar,
                 A_bar: A, B_bar: -B, C_bar: C, D_bar: -D}
psiR_scaled = expand(psi_scaled.subs(reversibility))

In [14]:
psi_quadratic_relations = []
psi_cubic_relations = []

invariant_quadratic_terms = []
invariant_cubic_terms = []

for psi_l, psi_r in zip(psi_scaled, psiR_scaled):
    tmp_coll = collect(psi_l - psi_r, [epsilon**2, epsilon**3], evaluate=False)
    tmp_quad = collect(tmp_coll[epsilon._sympy_()**2], quadratic_terms, evaluate=False)
    tmp_cubic = collect(tmp_coll[epsilon._sympy_()**3], cubic_terms, evaluate=False)
    for term in quadratic_terms:
        try:
            psi_quadratic_relations.append(Add(tmp_quad[term._sympy_()]))
        except KeyError:
            invariant_quadratic_terms.append(term)
    for term in cubic_terms:
        try:
            psi_cubic_relations.append(Add(tmp_cubic[term._sympy_()]))
        except KeyError:
            invariant_cubic_terms.append(term)

In [15]:
quad_sol_rev = {}
psi_quadratic_relations = list(set(psi_quadratic_relations))
for eqn in psi_quadratic_relations:
    if eqn == 0:
        continue
    tmp_var = list(eqn.free_symbols)[0]
    tmp_sol = linsolve([eqn], [tmp_var])[0]
    for i in range(len(psi_quadratic_relations)):
        psi_quadratic_relations[i] = psi_quadratic_relations[i].subs({tmp_var: tmp_sol})
    quad_sol_rev.update({tmp_var: tmp_sol})

In [16]:
cubic_sol_rev = {}
psi_cubic_relations = list(set(psi_cubic_relations))
for eqn in psi_cubic_relations:
    if eqn == 0:
        continue
    tmp_var = list(eqn.free_symbols)[0]
    tmp_sol = linsolve([eqn], [tmp_var])[0]
    for i in range(len(psi_cubic_relations)):
        psi_cubic_relations[i] = psi_cubic_relations[i].subs({tmp_var: tmp_sol})
    cubic_sol_rev.update({tmp_var: tmp_sol})

In [17]:
#for i, p in enumerate(poly):
#    poly[i] = expand(p.subs(quad_sol_rev))
#for i, p in enumerate(poly):
#    poly[i] = expand(p.subs(cubic_sol_rev))

#psi = Matrix(poly)

In [18]:
#quad_coeffs = [coeff for coeff in psi.free_symbols if coeff in quadratic_coefficients]
#quadratic_coefficients = quad_coeffs

In [19]:
#cubic_coeffs = [coeff for coeff in psi.free_symbols if coeff in cubic_coefficients]
#cubic_coefficients = cubic_coeffs

In [20]:
psi_A = Matrix([diff(p, A) for p in poly])
psi_B = Matrix([diff(p, B) for p in poly])
psi_C = Matrix([diff(p, C) for p in poly])
psi_D = Matrix([diff(p, D) for p in poly])
psi_A_bar = Matrix([diff(p, A_bar) for p in poly])
psi_B_bar = Matrix([diff(p, B_bar) for p in poly])
psi_C_bar = Matrix([diff(p, C_bar) for p in poly])
psi_D_bar = Matrix([diff(p, D_bar) for p in poly])

In [21]:
P1, P2, P3, P4 = symbols('P1 P2 P3 P4')
Q1, Q2, Q3, Q4 = symbols('Q1 Q2 Q3 Q4')
R1, R2, R3, R4 = symbols('R1 R2 R3 R4')
S1, S2, S3, S4 = symbols('S1 S2 S3 S4')
T1, T3, T7, T8, T9  = symbols('T1 T3 T7 T8 T9')
T2, T4, T5, T6, T10  = symbols('T2 T4 T5 T6 T10')
P1_bar, P2_bar, P3_bar, P4_bar = symbols('P1_bar P2_bar P3_bar P4_bar')
Q1_bar, Q2_bar, Q3_bar, Q4_bar = symbols('Q1_bar Q2_bar Q3_bar Q4_bar')
R1_bar, R2_bar, R3_bar, R4_bar = symbols('R1_bar R2_bar R3_bar R4_bar')
S1_bar, S2_bar, S3_bar, S4_bar = symbols('S1_bar S2_bar S3_bar S4_bar')
T1_bar, T3_bar, T7_bar, T8_bar, T9_bar  = symbols('T1_bar T3_bar T7_bar T8_bar T9_bar')
T2_bar, T4_bar, T5_bar, T6_bar, T10_bar  = symbols('T2_bar T4_bar T5_bar T6_bar T10_bar')
free_coefficients = Matrix([P1, P2, P3, P4,
                            Q1, Q2, Q3, Q4,
                            R1, R2, R3, R4,
                            S1, S2, S3, S4,
                            T1, T2, T3, T4,
                            T5, T6, T7, T8,
                            T9, T10,
                            P1_bar, P2_bar, P3_bar, P4_bar,
                            Q1_bar, Q2_bar, Q3_bar, Q4_bar,
                            R1_bar, R2_bar, R3_bar, R4_bar,
                            S1_bar, S2_bar, S3_bar, S4_bar,
                            T1_bar, T2_bar, T3_bar, T4_bar,
                            T5_bar, T6_bar, T7_bar, T8_bar,
                            T9_bar, T10_bar])

P = P1 * c1 + P2 * c3 + P3 * c5 + P4 * c7 + 3 * T1 * (c1 * w2 + c3 * w1) - I * T2 * c1 * w1
P = simplify(expand(P))
Q = Q1 * c1 + Q2 * c3 + Q3 * c5 + Q4 * c7 + T1 * (c3 * w2 - 4 * c5 * w1 + c1 * w1**2) - I * T2 * c3 * w1 - T3 * c1 * w1 - I * T4 * (c3 * w1 + c1 * w2)
Q = simplify(expand(Q))
R = R1 * c1 + R2 * c3 + R3 * c5 + R4 * c7 + T1 * (3 * c7 * w1 - c5 * w2 - 2 * c1 * w1 * w2 - 2 * c3 * w1**2) - I * T2 * (c5 * w1 - c1 * w1**2) - T3 * (c1 * w2 - c3 * w1) / 2 + I * T4 * (2 * c3 * w2 - 3 * c5 * w1 + 2 * c1 * w1**2) + I * T5 * c1 * w1 + I * T6 * (c3 * w2 - c5 * w1 + c1 * w1**2) - T7 * (c1 * w2 + c3 * w1) / 2
R = simplify(expand(R))
S = S1 * c1 + S2 * c3 + S3 * c5 + S4 * c7 + T1 * (2 * c1 * w2**2 - 3 * c7 * w2 + 2 * c3 * w1 * w2 + c5 * w1**2) - I * T2 * (c7 * w1 - c3 * w1**2) - T3 * (c3 * w2 - c1 * w1**2) / 2 + I * T4 * (4 * c1 * w1 * w2 + 4 * c3 * w1**2 - 3 * c5 * w2 - 6 * c7 * w1) - I * T5 * c1 * w2 + I * T6 * (2 * c1 * w1 * w2 + 2 * c3 * w1**2 - 2 * c5 * w2 - 3 * c7 * w1) + T7 * (c3 * w2 + c1 * w1**2) / 2 - T8 * c1 * w1 + T9 * (c3 * w2 - c5 * w1 + c1 * w1**2) + I * T10 * (c3 * w1 + c1 * w2)
S = simplify(expand(S))

P_bar = simplify(expand(conjugate(P).subs(relations)))
Q_bar = simplify(expand(conjugate(Q).subs(relations)))
R_bar = simplify(expand(conjugate(R).subs(relations)))
S_bar = simplify(expand(conjugate(S).subs(relations)))

In [22]:
assumptions = {conjugate(P1): P1_bar, conjugate(P2): P2_bar, conjugate(P3): P3_bar, conjugate(P4): P4_bar,
               conjugate(Q1): Q1_bar, conjugate(Q2): Q2_bar, conjugate(Q3): Q3_bar, conjugate(Q4): Q4_bar,
               conjugate(R1): R1_bar, conjugate(R2): R2_bar, conjugate(R3): R3_bar, conjugate(R4): R4_bar,
               conjugate(S1): S1_bar, conjugate(S2): S2_bar, conjugate(S3): S3_bar, conjugate(S4): S4_bar,
               conjugate(T1): T1_bar, conjugate(T2): T2_bar, conjugate(T3): T3_bar, conjugate(T4): T4_bar,
               conjugate(T5): T5_bar, conjugate(T6): T6_bar, conjugate(T7): T7_bar, conjugate(T8): T8_bar,
               conjugate(T9): T9_bar, conjugate(T10): T10_bar,
               conjugate(P1_bar): P1, conjugate(P2_bar): P2, conjugate(P3_bar): P3, conjugate(P4_bar): P4,
               conjugate(Q1_bar): Q1, conjugate(Q2_bar): Q2, conjugate(Q3_bar): Q3, conjugate(Q4_bar): Q4,
               conjugate(R1_bar): R1, conjugate(R2_bar): R2, conjugate(R3_bar): R3, conjugate(R4_bar): R4,
               conjugate(S1_bar): S1, conjugate(S2_bar): S2, conjugate(S3_bar): S3, conjugate(S4_bar): S4,
               conjugate(T1_bar): T1, conjugate(T2_bar): T2, conjugate(T3_bar): T3, conjugate(T4_bar): T4,
               conjugate(T5_bar): T5, conjugate(T6_bar): T6, conjugate(T7_bar): T7, conjugate(T8_bar): T8,
               conjugate(T9_bar): T9, conjugate(T10_bar): T10}

In [23]:
P = P.subs(assumptions)
Q = Q.subs(assumptions)
R = R.subs(assumptions)
S = S.subs(assumptions)
P_bar = P_bar.subs(assumptions)
Q_bar = Q_bar.subs(assumptions)
R_bar = R_bar.subs(assumptions)
S_bar = S_bar.subs(assumptions)

In [24]:
simplify(P)

(A*(A*A_bar*P1 + I*A*B_bar*P2 + A*C_bar*P3 + I*A*D_bar*P4 - I*A_bar*B*P2 + A_bar*C*P3 - 2*I*A_bar*C*T2 - I*A_bar*D*P4 - 9*I*A_bar*D*T1 - B*B_bar*P3 - I*B*C_bar*P4 + I*B_bar*C*P4 + 6*I*B_bar*C*T1) + I*A_bar*B**2*T2 + 3*I*A_bar*B*C*T1 - 3*I*B**2*B_bar*T1)/A

In [25]:
A_x = I * A + B + I * A * P
A_x = simplify(expand(A_x))
B_x = I * B + C + I * B * P + A * Q
B_x = simplify(expand(B_x))
C_x = I * C + D + I * C * P + B * Q + I * A * R
C_x = simplify(expand(C_x))
D_x = I * D + I * D * P + C * Q + I * B * R + A * S
D_x = simplify(expand(D_x))

A_bar_x = simplify(conjugate(A_x)).subs(relations).subs(assumptions)
B_bar_x = simplify(conjugate(B_x)).subs(relations).subs(assumptions)
C_bar_x = simplify(conjugate(C_x)).subs(relations).subs(assumptions)
D_bar_x = simplify(conjugate(D_x)).subs(relations).subs(assumptions)

In [26]:
A_x

I*A**2*A_bar*P1 - A**2*B_bar*P2 + I*A**2*C_bar*P3 - A**2*D_bar*P4 + A*A_bar*B*P2 + I*A*A_bar*C*P3 + 2*A*A_bar*C*T2 + A*A_bar*D*P4 + 9*A*A_bar*D*T1 - I*A*B*B_bar*P3 + A*B*C_bar*P4 - A*B_bar*C*P4 - 6*A*B_bar*C*T1 + I*A - A_bar*B**2*T2 - 3*A_bar*B*C*T1 + 3*B**2*B_bar*T1 + B

In [27]:
A_bar_x

-I*A*A_bar**2*P1_bar + A*A_bar*B_bar*P2_bar - I*A*A_bar*C_bar*P3_bar + 2*A*A_bar*C_bar*T2_bar + A*A_bar*D_bar*P4_bar + 9*A*A_bar*D_bar*T1_bar - A*B_bar**2*T2_bar - 3*A*B_bar*C_bar*T1_bar - A_bar**2*B*P2_bar - I*A_bar**2*C*P3_bar - A_bar**2*D*P4_bar + I*A_bar*B*B_bar*P3_bar - A_bar*B*C_bar*P4_bar - 6*A_bar*B*C_bar*T1_bar + A_bar*B_bar*C*P4_bar - I*A_bar + 3*B*B_bar**2*T1_bar + B_bar

In [28]:
# construct LHS of equation
LHS = (zeta_0 + psi_A) * A_x + (zeta_1 + psi_B) * B_x + (zeta_2 + psi_C) * C_x + (zeta_3 + psi_D) * D_x
LHS += (zeta_0_bar + psi_A_bar) * A_bar_x + (zeta_1_bar + psi_B_bar) * B_bar_x + (zeta_2_bar + psi_C_bar) * C_bar_x + (zeta_3_bar + psi_D_bar) * D_bar_x

LHS = expand(LHS).subs(scalings)

# truncate
#LHS = LHS + O(epsilon**4)

In [29]:
# construct RHS of equation
z = z_tilde + psi
Mz = M * z
Mz = Mz.subs(scalings)

# linear part
RHS = L * z
RHS = expand(RHS).subs(scalings)

# nonlinear part
n2, n3 = symbols('n2 n3', real=True)
eps_terms = collect(Mz[0], [epsilon, epsilon**2], evaluate=False)
Mz = epsilon * eps_terms[epsilon._sympy_()] + epsilon**2 * eps_terms[epsilon._sympy_()**2]

# quadratic
n2_terms = n2 * Mz**2
n2_terms = collect(expand(n2_terms), [epsilon, epsilon**2], evaluate=False)
n2_terms = epsilon**2 * n2_terms[epsilon._sympy_()**2] + epsilon**3 * n2_terms[epsilon._sympy_()**3]

# cubic
n3_terms = n3 * Mz**3
n3_terms = collect(expand(n3_terms), [epsilon, epsilon**2], evaluate=False)
n3_terms = epsilon**3 * n3_terms[epsilon._sympy_()**3]

N = Matrix([0, 0, 0, 0, 0, 0, 0, n2_terms + n3_terms])
RHS += N

## Quadratic terms

In [30]:
# quadratic solution
out_l = [el.coeff(epsilon, 2) for el in LHS]
out_r = [el.coeff(epsilon, 2) for el in RHS]
quadratic_equations = []
for l, r in zip(out_l, out_r):
    tmp = collect(expand(l - r), quadratic_terms, evaluate=False)
    for term in quadratic_terms:
        quadratic_equations.append(Add(tmp[term._sympy_()]))  # incompatibility between sympy and symengine symbols

In [31]:
quadratic_equations[1]

psi_010000010 - psi_110000001

In [32]:
# create the matrices
Q = []
Qr = []
for eq in quadratic_equations:
    rhs_term = 0
    Q.append([eq.coeff(coeff) for coeff in quadratic_coefficients])
    for coeff in eq.free_symbols.difference(set(quadratic_coefficients)):
        rhs_term += -coeff * eq.coeff(coeff) 
    Qr.append(rhs_term)
Q = Matrix(Q)
Qr = Matrix(Qr)

In [33]:
sol = Q.LUsolve(Qr)

In [34]:
quadratic_subs = {key: val for key, val in zip(quadratic_coefficients, sol)}
print(quadratic_subs)

{psi_000000002: (-352720/6561)*n2, psi_100000002: 1463840/19683*I*n2, psi_200000002: (1992920/19683)*n2, psi_300000002: -889840/6561*I*n2, psi_400000002: (-3518320/19683)*n2, psi_500000002: 4564000/19683*I*n2, psi_600000002: (647900/2187)*n2, psi_700000002: -7344200/19683*I*n2, psi_000000011: -652480/19683*I*n2, psi_100000011: (-934760/19683)*n2, psi_200000011: 1316320/19683*I*n2, psi_300000011: (1820720/19683)*n2, psi_400000011: -2472640/19683*I*n2, psi_500000011: (-3296900/19683)*n2, psi_600000011: 4318000/19683*I*n2, psi_700000011: (5560400/19683)*n2, psi_000000020: (12340/2187)*n2, psi_100000020: -18440/2187*I*n2, psi_200000020: (-27064/2187)*n2, psi_300000020: 38960/2187*I*n2, psi_400000020: (54946/2187)*n2, psi_500000020: -75860/2187*I*n2, psi_600000020: (-102520/2187)*n2, psi_700000020: 135728/2187*I*n2, psi_000000101: (49360/6561)*n2, psi_100000101: -73760/6561*I*n2, psi_200000101: (-108256/6561)*n2, psi_300000101: 155840/6561*I*n2, psi_400000101: (219784/6561)*n2, psi_50000010

In [35]:
key = list(quadratic_subs.keys())[0]
print(key, quad_sol_rev[key], quadratic_subs[key], quadratic_subs[quad_sol_rev[key]])

psi_000000002 psi_000020000 (-352720/6561)*n2 (-352720/6561)*n2


## Cubic terms

The matrix represenbting the LHS of the system of equations at cubic order is rank-deficient, so we can not use one of the solve procedures. Instead we calculate the null space of the LHS and multiply by the RHS to recover a system of equations involving the coefficients {P1, P2, Q1, Q2} (and their conjugates).

In [36]:
# cubic solution
out_l = [el.coeff(epsilon, 3) for el in LHS]
out_r = [el.coeff(epsilon, 3) for el in RHS]
cubic_equations = []
for l, r in zip(out_l, out_r):
    tmp = collect(expand(l - r).subs(quadratic_subs), cubic_terms, evaluate=False)
    for term in cubic_terms:
        cubic_equations.append(Add(tmp[term._sympy_()]))  # incompatibility betwen sympy and symengine symbols

In [37]:
cubic_equations[0]

I*psi_000020001 + psi_000020010 + psi_000110001 - psi_100020001

In [38]:
# create the matrices
C3 = []
C3r = []
for eq in cubic_equations:
    rhs_term = 0
    C3.append([eq.coeff(coeff) for coeff in cubic_coefficients])
    n_coeffs = eq.free_symbols.difference(set(cubic_coefficients))
    coll = {k: v for k, v in collect(eq, n_coeffs, evaluate=False).items() if k != 1}
    if len(coll) > 0:
        for k, v in coll.items():
            rhs_term += -k * v
    C3r.append(rhs_term)
C3 = Matrix(C3)
C3r = Matrix(C3r)

In [39]:
C3r[959]

21*I*P4 - 35*I*Q4 + 84*I*T1 + 70*T4 + 70*T6

In [40]:
QQ = C3._sympy_()
QQr = C3r._sympy_()

In [41]:
QT = QQ.T

In [ ]:
kern = QT.nullspace()

In [ ]:
M_kern = Matrix([list(k) for k in kern])

In [ ]:
system = simplify(M_kern * C3r)

In [ ]:
sol = solve(system, free_coefficients)

In [ ]:
for k, v in sol.items():
    disp.display(k, v)